In [1]:
source extend_bashkernel.source

Ready.


In [ ]:
# (1) optional reset of Wakame-vdc
# Comment out the following lines and run cell to remove all
# added images and instances.

reset-wakame.sh
#.

In [2]:
# (2) Create Centos instance using mussel

cat <<EOS >vifs.json
{
 "eth0":{"network":"nw-demo1","security_groups":"sg-cicddemo"}
}
EOS

mussel instance create --cpu-cores 2 --hypervisor openvz \
    --image-id wmi-centos1d64 --memory-size 2048 \
    --ssh-key-id ssh-cicddemo --vifs vifs.json --display-name centos
#.

---
:id: i-urv3fm2w
:account_id: a-shpoolxx
:host_node: 
:cpu_cores: 2
:memory_size: 2048
:arch: x86_64
:image_id: wmi-centos1d64
:created_at: 2016-02-03 06:54:11.000000000 Z
:updated_at: 2016-02-03 06:54:11.543380546 Z
:terminated_at: 
:deleted_at: 
:state: scheduling
:status: init
:ssh_key_pair:
  :uuid: ssh-cicddemo
  :display_name: cicddemo
:volume:
- :vol_id: vol-4n6svhs7
  :state: scheduling
:vif: []
:hostname: urv3fm2w
:ha_enabled: 0
:hypervisor: openvz
:display_name: centos
:service_type: std
:monitoring:
  :enabled: false
  :mail_address: []
  :items: {}
:labels:
- :resource_uuid: i-urv3fm2w
  :name: monitoring.enabled
  :value_type: 1
  :value: 'false'
  :created_at: 2016-02-03 06:54:11.000000000 Z
  :updated_at: 2016-02-03 06:54:11.000000000 Z
:boot_volume_id: vol-4n6svhs7


In [4]:
# (teststep) Wait for instance's ssh port to become active
: ${IP:=10.0.2.100}

# echo
#.

# (teststep) Wait for instance's ssh port to become active
: ${IP:=10.0.2.100}

# echo


## repoファイルのインストール

In [5]:
ssh -qi ../mykeypair root@10.0.2.100 curl -fSkL http://pkg.jenkins-ci.org/redhat/jenkins.repo -o /etc/yum.repos.d/jenkins.repo
#.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    75    0    75    0     0    114      0 --:--:-- --:--:-- --:--:--   436


## 公開鍵のインポート

In [6]:
ssh -qi ../mykeypair root@10.0.2.100 rpm --import http://pkg.jenkins-ci.org/redhat/jenkins-ci.org.key
#.

## java実行環境のインストール

In [7]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y java-1.7.0-openjdk
#.

Loaded plugins: fastestmirror
Setting up Install Process
Determining fastest mirrors
 * base: www.ftp.ne.jp
 * extras: www.ftp.ne.jp
 * updates: www.ftp.ne.jp
Resolving Dependencies
--> Running transaction check
---> Package java-1.7.0-openjdk.x86_64 1:1.7.0.95-2.6.4.0.el6_7 will be installed
--> Processing Dependency: libjpeg = 6b for package: 1:java-1.7.0-openjdk-1.7.0.95-2.6.4.0.el6_7.x86_64
--> Processing Dependency: jpackage-utils >= 1.7.3-1jpp.2 for package: 1:java-1.7.0-openjdk-1.7.0.95-2.6.4.0.el6_7.x86_64
--> Processing Dependency: xorg-x11-fonts-Type1 for package: 1:java-1.7.0-openjdk-1.7.0.95-2.6.4.0.el6_7.x86_64
--> Processing Dependency: tzdata-java for package: 1:java-1.7.0-openjdk-1.7.0.95-2.6.4.0.el6_7.x86_64
--> Processing Dependency: libpulse.so.0(PULSE_0)(64bit) for package: 1:java-1.7.0-openjdk-1.7.0.95-2.6.4.0.el6_7.x86_64
--> Processing Dependency: libpng12.so.0(PNG12_0)(64bit) for package: 1:java-1.7.0-openjdk-1.7.0.95-2.6.4.0.el6_7.x86_64
--> Processing Dependen

## jenkins coreのインストール

In [8]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y jenkins
#.

Loaded plugins: fastestmirror
Setting up Install Process
Loading mirror speeds from cached hostfile
 * base: www.ftp.ne.jp
 * extras: www.ftp.ne.jp
 * updates: www.ftp.ne.jp
Resolving Dependencies
--> Running transaction check
---> Package jenkins.noarch 0:1.646-1.1 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package           Arch             Version             Repository         Size
Installing:
 jenkins           noarch           1.646-1.1           jenkins            60 M

Transaction Summary
Install       1 Package(s)

Total download size: 60 M
Installed size: 60 M
Running rpm_check_debug
Running Transaction Test
Transaction Test Succeeded
Running Transaction
  Installing : jenkins-1.646-1.1.noarch                                     1/1 
  Verifying  : jenkins-1.646-1.1.noarch                                     1/1 

Installed:
  jenkins.noarch 0:1.646-1.1                                                    

Complete!


## 関連するビルドツールのインストール

In [9]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y \
    git \
    iputils nc \
    qemu-kvm qemu-img \
    parted kpartx \
    rpm-build automake createrepo \
    openssl-devel zlib-devel readline-devel \
    gcc
#.

Loaded plugins: fastestmirror
Setting up Install Process
Loading mirror speeds from cached hostfile
 * base: www.ftp.ne.jp
 * extras: www.ftp.ne.jp
 * updates: www.ftp.ne.jp
Resolving Dependencies
--> Running transaction check
---> Package automake.noarch 0:1.11.1-4.el6 will be installed
--> Processing Dependency: perl >= 5.006 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl >= 5.005_03 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl >= 5.005 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl >= 5.000 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: autoconf >= 2.62 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl(vars) for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl(strict) for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl(constant) for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl(POSIX) for p

## sudo設定

### sudo権限の付与


In [10]:
ssh -qi ../mykeypair root@10.0.2.100 "echo 'jenkins ALL=(ALL) NOPASSWD: ALL' >>/etc/sudoers"
ssh -qi ../mykeypair root@10.0.2.100 grep jenkins /etc/sudoers
#.

jenkins ALL=(ALL) NOPASSWD: ALL


### tty設定の変更

In [11]:
ssh -qi ../mykeypair root@10.0.2.100 'sed -i "s/^\(^Defaults\s*requiretty\).*/# \1/" /etc/sudoers'
#.

### Loosen Java Security so Jenkins Plugins will work

In [12]:
# see https://issues.jenkins-ci.org/browse/JENKINS-31089
javadir=/usr/lib/jvm/java-1.7.0-openjdk-1.7.0.95.x86_64/
secfile=jre/lib/security/java.security
ssh -qi ../mykeypair root@10.0.2.100 \
   "sudo sed -i 's/, MD5, RSA keySize < 1024//' $javadir/$secfile"
#.


## jenkinsの起動


In [13]:
ssh -qi ../mykeypair root@10.0.2.100 service jenkins start
#.